# Webscraping the table

### import all the necessary libraries

In [4]:
pip install geopandas


Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install geopy
from geopy.geocoders import Nominatim


In [6]:
import requests
import pandas as pd
import csv
import urllib.request
from bs4 import BeautifulSoup



In [63]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


### Define url and request

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response.status_code)

200


### parse html

In [12]:
soup = BeautifulSoup(response.text,"html.parser")
table = soup.findAll('table',{"class":"sortable"})[0]
tr = table.findAll(['tr'])

### create CSV file

In [13]:
csvFile = open("postalcodes.csv",'wt',newline='', encoding='utf-8')

writer = csv.writer(csvFile)
for cell in tr:
            th = cell.find_all("th")
            th_data = [col.text.strip('\n') for col in th]
            td = cell.find_all("td")
            row = [i.text.replace('\n','') for i in td]
            writer.writerow(th_data+row)      



In [14]:
df=pd.read_csv('postalcodes.csv')
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### drop all 'not assigned'

In [46]:
indexNames = df[ df['Borough'] == 'Not assigned' ].index


In [16]:
df.drop(indexNames, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### shape

In [18]:
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [19]:
df.shape

(103, 3)

### getting coordinates and make a new dataframe 

In [20]:
latitude=[]
longitude=[]
postal_code=df['Postal Code']
#for code in adress: 
#    g = geocode.arcgis('{}, Toronto, Ontario'.format(code))
#    print(code, g.latlng)
#   while (g.latlng is None):
#        g = geocode.arcgis('{}, Toronto, Ontario'.format(code))
#        print(code, g.latlng)
#   latlng = g.latlng
#    latitude.append(latlng[0])
#    longitude.append(latlng[1])
    
    

# initialize your variable to None
g=None
geolocator = Nominatim(user_agent="foursquare_agent")
# loop until you get the coordinates
for code in df['Postal Code']:
    g = geolocator.geocode('{}, Toronto, Ontario'.format(code))
   
    if (g is None):
        g = geolocator.geocode('{}, Toronto, Ontario'.format(code))
       # print(code, g)
        latitude.append('Not available')
        longitude.append('Not available')
    else: 
       # print(code, g.latitude, g.longitude)
        latitude.append(g.latitude)
        longitude.append(g.longitude)
        

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.6535,-79.3839
1,M4A,North York,Victoria Village,Not available,Not available
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",Not available,Not available
3,M6A,North York,"Lawrence Manor, Lawrence Heights",Not available,Not available
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6535,-79.3839


In [21]:
df_geo=pd.read_csv('Geospatial_Coordinates.csv')

In [23]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df['Latitude']=df_geo['Latitude']
df['Longitude']=df_geo['Longitude']
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


In [66]:
# create map of Toronto using latitude and longitude values

toronto_location=[43.651070, -79.347015]
map_toronto = folium.Map(location=toronto_location, zoom_start=10)

 #add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'],df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


### Only use toronto boroughs

In [26]:
toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


In [50]:
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'],toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [71]:
#hide cell 
CLIENT_ID = 'IMQ4MXE0FYSCDIUKACLFBUYHC5HGN5I220PZ3ZB4TMCEQH4R' # your Foursquare ID
CLIENT_SECRET = 'L5Y0O4WEZ2VXMTNVRRNQKQFGKXAY021QV4JP3F12NPIYJFYH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [28]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [29]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


In [30]:
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET, 
    neighbourhood_latitude,
    neighbourhood_longitude,
    VERSION,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=IMQ4MXE0FYSCDIUKACLFBUYHC5HGN5I220PZ3ZB4TMCEQH4R&client_secret=L5Y0O4WEZ2VXMTNVRRNQKQFGKXAY021QV4JP3F12NPIYJFYH&ll=43.7635726,-79.1887115&v=20180605&radius=500&limit=100'

In [47]:
results = requests.get(url).json()


In [48]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

<ipython-input-34-6a7f8778540a>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406
5,Woburn Medical Centre,Medical Center,43.766631,-79.192286
6,Lawrence Ave E & Kingston Rd,Intersection,43.767704,-79.189490
7,Eggsmart,Breakfast Spot,43.767800,-79.190466


In [49]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [52]:
print(toronto_venues.shape)
toronto_venues.head()

(152, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [53]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,1,1,1,1,1,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,5,5,5,5,5,5
Christie,2,2,2,2,2,2
Church and Wellesley,6,6,6,6,6,6
"Commerce Court, Victoria Hotel",2,2,2,2,2,2
"First Canadian Place, Underground city",1,1,1,1,1,1
"Garden District, Ryerson",4,4,4,4,4,4
"Harbourfront East, Union Station, Toronto Islands",8,8,8,8,8,8


In [54]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 85 uniques categories.


In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,...,Supplement Shop,Sushi Restaurant,Tanning Salon,Thai Restaurant,Thrift / Vintage Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

toronto_grouped.shape

(19, 86)

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0  Martial Arts School   1.0
1              Airport   0.0
2   Light Rail Station   0.0
3             Pharmacy   0.0
4            Pet Store   0.0


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0         Garden Center  0.06
1         Burrito Place  0.06
2        Farmers Market  0.06
3  Fast Food Restaurant  0.06
4      Recording Studio  0.06


----Central Bay Street----
           venue  freq
0    Pizza Place   0.2
1       Pharmacy   0.2
2  Grocery Store   0.2
3    Coffee Shop   0.2
4        Butcher   0.2


----Christie----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5
2            Airport   0.0
3          Pet Store   0.0
4      Movie Theater   0.0


----Church and Wellesley----
                venue  freq
0      Discount Store  0.17
1      Sandwich Place  0.17
2        Intersection  0.17
3         Coffee S

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
import numpy as np 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts School,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Garden Center,Brewery,Light Rail Station,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place
2,Central Bay Street,Pharmacy,Butcher,Pizza Place,Coffee Shop,Grocery Store,Bank,Fish & Chips Shop,Dessert Shop,Diner,Discount Store
3,Christie,Park,Food & Drink Shop,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
4,Church and Wellesley,Pizza Place,Coffee Shop,Chinese Restaurant,Discount Store,Sandwich Place,Intersection,Yoga Studio,Falafel Restaurant,Dance Studio,Dessert Shop
5,"Commerce Court, Victoria Hotel",Playground,Trail,Yoga Studio,Farmers Market,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store
6,"First Canadian Place, Underground city",Baseball Field,Yoga Studio,Fish & Chips Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
7,"Garden District, Ryerson",General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Farmers Market,Dance Studio,Dessert Shop,Diner,Discount Store
8,"Harbourfront East, Union Station, Toronto Islands",Skating Rink,Curling Ice,Athletics & Sports,Dance Studio,Beer Store,Park,Intersection,Fast Food Restaurant,Dessert Shop,Diner
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Pub,Italian Restaurant,Sushi Restaurant,Pizza Place,Fish & Chips Shop,Latin American Restaurant,Indie Movie Theater,Health Food Store


In [59]:
from sklearn.cluster import KMeans 
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 4, 1, 3, 0, 1, 1, 1], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,1,Mexican Restaurant,Intersection,Medical Center,Bank,Rental Car Location,Restaurant,Breakfast Spot,Electronics Store,Dance Studio,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,1,Athletics & Sports,Gas Station,Bakery,Bank,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Yoga Studio,Farmers Market
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,1,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Farmers Market,Dance Studio,Dessert Shop,Diner,Discount Store
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,1,Fast Food Restaurant,Gym,Breakfast Spot,Grocery Store,Indian Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Pharmacy
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,2,Martial Arts School,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market


In [69]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=toronto_location, zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,0,Construction & Landscaping,Baseball Field,Fish & Chips Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
17,Downtown Toronto,0,Baseball Field,Yoga Studio,Fish & Chips Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
